<a href="https://colab.research.google.com/github/kazzastic/BreastCancer-Deep-Learning/blob/master/Patches_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [209]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (227,227,3), pooling = 'avg')
x = base_model.output
preds = Dense(3, activation='softmax', kernel_regularizer=l2(0.01))(x)
model = Model(inputs=base_model.input, outputs=preds)
top_layer_nb = 15
class_weight = {0:1.0, 1:pos_cls_weight, 2:neg_cls_weight} 

In [0]:
org_model = model

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer = ModelCheckpoint(filepath='Patch_CNN.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

#ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, partience = 2, verbose = 1)

#setting the learning rate
Adam = Adam(learning_rate=0.001)
SGD = SGD(learning_rate=0.001)

# First Stage Of training with learning rate 0.001

In [0]:
pos_cls_weight, neg_cls_weight = 1., 1.
for layer in org_model.layers[:-1]:
  layer.trainable = False
#class_weight = { 0:1.0, 1:pos_cls_weight, 2:neg_cls_weight }
nb_worker = 1
if nb_worker == 1:
  pickle_safe = False
else:
  pickle_safe = True


In [0]:
train_data_dir = "/content/drive/My Drive/patches_sort"

In [217]:
datagen = ImageDataGenerator(validation_split=0.3)
batch_size = 32
img_width , img_height = 227, 227

# Retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'validation')

Found 40530 images belonging to 3 classes.
Found 17367 images belonging to 3 classes.


In [218]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam,
              metrics=['accuracy'],
              )
model.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

Run the cell below for about 3 to 4 epoch

In [219]:
hist = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=3,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer, early_stopping],
        verbose=1,
        class_weight = class_weight)

Epoch 1/3
32/32 [==============================] - 71s 2s/step - loss: 1.3610 - acc: 0.3848

Epoch 00001: val_acc improved from -inf to 0.38477, saving model to Patch_CNN.h5
32/32 [==============================] - 81s 3s/step - loss: 1.6031 - acc: 0.3779 - val_loss: 1.3610 - val_acc: 0.3848
Epoch 2/3
32/32 [==============================] - 6s 199ms/step - loss: 1.2597 - acc: 0.3789

Epoch 00002: val_acc did not improve from 0.38477
32/32 [==============================] - 10s 322ms/step - loss: 1.3189 - acc: 0.4004 - val_loss: 1.2597 - val_acc: 0.3789
Epoch 3/3
32/32 [==============================] - 5s 160ms/step - loss: 1.2292 - acc: 0.3760

Epoch 00003: val_acc did not improve from 0.38477
32/32 [==============================] - 9s 284ms/step - loss: 1.2934 - acc: 0.3867 - val_loss: 1.2292 - val_acc: 0.3760


In [0]:
loss_history = hist.history['val_loss']
acc_history = hist.history['val_acc']

# Second Stage of training with learning rate of 0.0001

In [221]:
Adam1 = tf.keras.optimizers.Adam(learning_rate = 0.0001)
for layer in org_model.layers[top_layer_nb:]:
  layer.trainable = True
model.compile(loss='categorical_crossentropy',
              optimizer=Adam1,
              metrics=['accuracy'],
              )
model.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer1 = ModelCheckpoint(filepath='2ndPatch_CNN.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping1 = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

In [223]:
hist = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=len(loss_history)+10,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer1, early_stopping1],
        verbose=1,
        initial_epoch = len(loss_history),
        class_weight = mod)

Epoch 4/13
32/32 [==============================] - 5s 159ms/step - loss: 1.0941 - acc: 0.4102

Epoch 00004: val_acc improved from -inf to 0.41016, saving model to 2ndPatch_CNN.h5
32/32 [==============================] - 12s 369ms/step - loss: 1.1646 - acc: 0.4189 - val_loss: 1.0941 - val_acc: 0.4102
Epoch 5/13
32/32 [==============================] - 5s 150ms/step - loss: 1.1091 - acc: 0.4268

Epoch 00005: val_acc improved from 0.41016 to 0.42676, saving model to 2ndPatch_CNN.h5
32/32 [==============================] - 8s 254ms/step - loss: 1.1107 - acc: 0.3965 - val_loss: 1.1091 - val_acc: 0.4268
Epoch 6/13
32/32 [==============================] - 5s 168ms/step - loss: 1.1565 - acc: 0.4365

Epoch 00006: val_acc improved from 0.42676 to 0.43652, saving model to 2ndPatch_CNN.h5
32/32 [==============================] - 9s 277ms/step - loss: 1.0664 - acc: 0.4482 - val_loss: 1.1565 - val_acc: 0.4365
Epoch 7/13
32/32 [==============================] - 5s 148ms/step - loss: 1.3513 - acc: 0.

In [0]:
import numpy as np
loss_history = np.append(loss_history, hist.history['val_loss'])
acc_history = np.append(acc_history, hist.history['val_acc'])

In [0]:
for layers in org_model.layers[:top_layer_nb]:
  layer.trainable = True

In [232]:
Adam2 = tf.keras.optimizers.Adam(learning_rate = 0.00001)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam2,
              metrics=['accuracy'],
              )
model.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [0]:
mod = tf.keras.models.load_model("/content/Patch_CNN.h5")
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer1 = ModelCheckpoint(filepath='3rdPatch_CNN.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping1 = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

In [208]:
hist = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=len(loss_history)+37,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer1, early_stopping1],
        verbose=1,
        initial_epoch = len(loss_history),
        class_weight = class_weight)

Epoch 14/50
32/32 [==============================] - 5s 157ms/step - loss: 2.0147 - acc: 0.5176

Epoch 00014: val_acc improved from -inf to 0.51758, saving model to 3rdPatch_CNN.h5
32/32 [==============================] - 12s 365ms/step - loss: 1.0167 - acc: 0.4697 - val_loss: 2.0147 - val_acc: 0.5176
Epoch 15/50
32/32 [==============================] - 5s 149ms/step - loss: 2.3738 - acc: 0.5225

Epoch 00015: val_acc improved from 0.51758 to 0.52246, saving model to 3rdPatch_CNN.h5
32/32 [==============================] - 8s 253ms/step - loss: 0.9951 - acc: 0.5029 - val_loss: 2.3738 - val_acc: 0.5225
Epoch 16/50
32/32 [==============================] - 5s 149ms/step - loss: 2.5427 - acc: 0.5186

Epoch 00016: val_acc did not improve from 0.52246
32/32 [==============================] - 8s 245ms/step - loss: 1.0206 - acc: 0.4736 - val_loss: 2.5427 - val_acc: 0.5186
Epoch 17/50
32/32 [==============================] - 5s 148ms/step - loss: 2.5628 - acc: 0.5303

Epoch 00017: val_acc improv